<a href="https://colab.research.google.com/github/pinilDissanayaka/RAG-Application-for-The-Constitution-of-the-Democratic-Socialist-Republic-of-Sri-Lanka/blob/main/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -r requirements.txt

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_6

In [16]:
import os
from langchain_groq import ChatGroq
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import Pinecone as PineconeVectorStore
from pinecone import ServerlessSpec, Pinecone
from langchain.storage import InMemoryByteStore
from google.colab import userdata
from langchain.embeddings import HuggingFaceEmbeddings

In [4]:
os.environ['PINECONE_API_KEY'] = userdata.get('PINECORN_API_KEY')
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
os.environ['LANGCHAIN_API_KEY']=userdata.get('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT']=userdata.get('LANGCHAIN_PROJECT')
os.environ['LANGCHAIN_TRACING_V2']="TRUE"

In [5]:
pdfLoader=PyPDFLoader(file_path="/content/srl132846.pdf")

docs=pdfLoader.load()

In [ ]:
docs[5]

Document(metadata={'source': '/content/srl132846.pdf', 'page': 5}, page_content='The Constitution of the Democratic Socialist Republic of Sri Lankavi\n20 Use of National languages in Parliament, Provincial Councils, and \nLocal Authorities  .... .... .... .... .... .... .... .... .... .... 10\n21 Medium of instruction  .... .... .... .... .... .... .... .... .... 10\n22 Languages of administration ....  ....  ....  ....  ....  ....  ....  ....  11\n23 Language of legislation ....  ....  ....  ....  ....  ....  ....  ....  ....  13\n24 Languages of the courts ...   ....   ....   ....   ....   ....   ....   ....   ....   13\n25 Provision for adequate facilities for use of languages provided for in \nthis Chapter  .... .... .... .... .... .... .... .... .... .... ....15\n25 A Provision of any law inconsistent with this Chapter deemed to be \nrepealed ....  ....  ....  ....  ....  ....  ....  ....  ....  ....  ....  ....  15\nCHAPTER - V\nCITIZENSHIP\n26 Citizenship of Sri Lanka  ..   ....

In [6]:
docsWithOutMetadata=list()

for doc in docs:
  docsWithOutMetadata.append(doc.page_content)

In [7]:
docsWithOutMetadata[5]

'The Constitution of the Democratic Socialist Republic of Sri Lankavi\n20 Use of National languages in Parliament, Provincial Councils, and \nLocal Authorities  .... .... .... .... .... .... .... .... .... .... 10\n21 Medium of instruction  .... .... .... .... .... .... .... .... .... 10\n22 Languages of administration ....  ....  ....  ....  ....  ....  ....  ....  11\n23 Language of legislation ....  ....  ....  ....  ....  ....  ....  ....  ....  13\n24 Languages of the courts ...   ....   ....   ....   ....   ....   ....   ....   ....   13\n25 Provision for adequate facilities for use of languages provided for in \nthis Chapter  .... .... .... .... .... .... .... .... .... .... ....15\n25 A Provision of any law inconsistent with this Chapter deemed to be \nrepealed ....  ....  ....  ....  ....  ....  ....  ....  ....  ....  ....  ....  15\nCHAPTER - V\nCITIZENSHIP\n26 Citizenship of Sri Lanka  ..   ....   ....   ....   ....   ....   ....   ....   ....   16\nCHAPTER - VI\nDIRECTIVE 

In [9]:
parentChunks=RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""]
).split_text(text="\n\n".join(docsWithOutMetadata))


childChunks=RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""]).split_text(text="\n\n".join(docsWithOutMetadata))

In [10]:
llm=ChatGroq(model="gemma2-9b-it",
             temperature=0.7,
             verbose=True)

In [11]:
parentStore=InMemoryByteStore()

In [12]:
pinecone=Pinecone()

In [14]:
INDEX_NAME="constitution"
DIMENSIONS=384

In [17]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [20]:
try:
  if not INDEX_NAME in pinecone.list_indexes().names():
    pinecone.create_index(
        name=INDEX_NAME,
        dimension=DIMENSIONS,
        metric="cosine",
        spec=ServerlessSpec(cloud='aws',
                          region='us-east-1')
    )
    print(f"Created {INDEX_NAME}")
  else:
    print(f"{INDEX_NAME} already exists.")
except Exception as e:
  print(e)

Created constitution
